# Dataset Preprocessing
In this notebook we are going to preprocess the slides to generate our sets of patches for training, validation, and testing.

In [1]:
from pathgen.utils.seeds import set_seed

global_seed = 123
set_seed(global_seed)

In [2]:
from pathgen.utils.paths import project_root

experiment_name = "all"
experiment_root = project_root() / "experiments" / experiment_name

## 1. Audit the dataset
Let's find out how many patches there are in each slide in the Camelyon16 dataset, so we can select from the in a principled way.

In [3]:
import pathgen.data.datasets.camelyon16 as camelyon16

train = camelyon16.training()
len(train)

270

In [4]:
train[0]

(PosixPath('/home/ubuntu/pathgen/data/camelyon16/raw/training/tumor/tumor_001.tif'),
 PosixPath('/home/ubuntu/pathgen/data/camelyon16/raw/training/lesion_annotations/tumor_001.xml'),
 'tumor',
 '')

In [5]:
from pathgen.preprocess.patching import make_index
from pathgen.preprocess.tissue_detection import TissueDetectorOTSU
from pathgen.preprocess.patching import GridPatchFinder

tissue_detector = TissueDetectorOTSU()
patch_finder = GridPatchFinder(6, 0, 256, 256)

index = make_index(train, tissue_detector, patch_finder)
index.summary()

indexing tumor_001.tif
indexing tumor_002.tif
indexing tumor_003.tif
indexing tumor_004.tif
indexing tumor_005.tif
indexing tumor_006.tif
indexing tumor_007.tif
indexing tumor_008.tif
indexing tumor_009.tif
indexing tumor_010.tif
indexing tumor_011.tif
indexing tumor_012.tif
indexing tumor_013.tif
indexing tumor_014.tif
indexing tumor_015.tif
indexing tumor_016.tif
indexing tumor_017.tif
indexing tumor_018.tif
indexing tumor_019.tif
indexing tumor_020.tif
indexing tumor_021.tif
indexing tumor_022.tif
indexing tumor_023.tif
indexing tumor_024.tif
indexing tumor_025.tif
indexing tumor_026.tif
indexing tumor_027.tif
indexing tumor_028.tif
indexing tumor_029.tif
indexing tumor_030.tif
indexing tumor_031.tif
indexing tumor_032.tif
indexing tumor_033.tif
indexing tumor_034.tif
indexing tumor_035.tif
indexing tumor_036.tif
indexing tumor_037.tif
indexing tumor_038.tif
indexing tumor_039.tif
indexing tumor_040.tif
indexing tumor_041.tif
indexing tumor_042.tif
indexing tumor_043.tif
indexing tu

AttributeError: 'DataFrameGroupBy' object has no attribute 'labels'

In [7]:
dir(index)

['__abstractmethods__',
 '__args__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__extra__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next_in_mro__',
 '__orig_bases__',
 '__origin__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__tree_hash__',
 '__weakref__',
 '_abc_cache',
 '_abc_generic_negative_cache',
 '_abc_generic_negative_cache_version',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_gorg',
 'count',
 'dataset',
 'index',
 'load',
 'patches',
 'save',
 'summary']

In [ ]:
training_summary = index.summary()
total_normal = training_summary['normal'].sum()
total_tumor = training_summary['tumor'].sum()
print(f"Total normal patches: {total_normal}")
print(f"Total tumor patches: {total_tumor}")

## 2. Sampling at a slide level
We want the samples in the different sets (train, validate, and test) to come from seperate slides. Test is it's own dataset, but validate has to be broken off from train on a per-slide basis. We want the split to take into account the number of patches in each set (because it is the patches that we are interested in). Let's base the split so that the validate set has about 30% of the tumor patches in it. Baseing it on the tumor patches makes sense because they are the class with the smallest number of samples.

Our algorithm for splitting will work like so:
1. Work out the total number of tumor patches in the whole dataset.
2. Work out what 30% (or whatever the amount is) of the total patches is.
3. Start counting with 0 patches and 0 slides in the validation set.
4. Randomly select a slide in training and move it to validation.
5. Add the number of patches in that slide to the total.
6. Repeat from 4 until the total number of patches is greater than 30% of the total.

In [ ]:
total_tumor = training_summary['tumor'].sum()
valid_tumor_count = int(total_tumor * 0.3)  # rounds down
valid_tumor_count

In [ ]:
import random

train_slide_indices = list(range(len(training_summary)))
random.shuffle(train_slide_indices)
valid_slide_indices = []
total_valid_patches = 0
while total_valid_patches < valid_tumor_count:
    slide_idx = train_slide_indices[0]
    tumor_count = training_summary.iloc[slide_idx]['tumor']
    total_valid_patches += tumor_count
    train_slide_indices.pop(0)
    valid_slide_indices.append(slide_idx)
    
# print them out
print(train_slide_indices)
print(valid_slide_indices)

# check there are no duplicates
print(len(train_slide_indices) + len(valid_slide_indices))
print(len(set(train_slide_indices + valid_slide_indices)))

Now we have the indices of the two sets, let's turn them into their own datasets based on those indices.

In [ ]:
len(train_slide_indices), len(valid_slide_indices)